In [1]:
import salabim as sim

In [85]:
class TestMachine(sim.Component):
    def __init__(self, machine_id, env, *args, **kwargs):
        super().__init__(env=env, *args, **kwargs)
        self.machine_id = machine_id
        self.buffer = sim.Queue(env=env)
    
    def process(self):
        while True:
            if len(self.buffer) == 0:
                yield self.passivate()
            job = self.buffer.pop()
            proc_time = job.proc_time
            print(f"proc time of job with ID {job.job_id} on machine ID {self.machine_id} is {proc_time}")
            yield self.hold(job.proc_time)
            job.activate()

class TestJob(sim.Component):
    def __init__(self, ident, machine_list, machine_dict, operation_procs, env, dispatcher, *args, **kwargs):
        super().__init__(env=env, *args, **kwargs)
        #self.machine_id = machine_id
        self.machine_list = machine_list.copy()
        self.proc_time = None
        self.counter = 0
        self.operation_procs = operation_procs.copy()
        self.machine_dict = machine_dict.copy()
        self.dict_keys = list(machine_dict.keys())
        self.job_id = ident
        self.dispatcher = dispatcher
        
    def process(self):
        while len(self.operation_procs) != 0:
            #print(f"Job-ID {self.job_id} starts {env.now()}")
            print(f"Job-ID {self.job_id} \t Operation with ID {self.counter}")
            self.proc_time = self.operation_procs.pop(0)
            #machine = machine_list.pop(0)
            #machine = self.machine_dict[self.dict_keys[self.counter]]
            machine = self.dispatcher.get_machine_instance_by_id(self.dict_keys[self.counter])
            #print(f"Machine instance is {machine}")
            #print(f"Machine instance is: {machine} and ispassive {machine.ispassive()}")
            self.enter(machine.buffer)
            if machine.ispassive():
                machine.activate()
            yield self.passivate()
            print(f"Time now Job-ID {self.job_id} is {self.env.now()}")
            self.counter += 1
        

In [86]:
class TestDispatcher():
    def __init__(self) -> None:
        pass
    
    def add_dicts(self, job_dict, machine_dict):
        self.job_dict = job_dict
        self.machine_dict = machine_dict
        
    def get_machine_instance_by_id(self, ident):
        return self.machine_dict[ident]

In [87]:
test_dispatcher = TestDispatcher()

In [90]:
env = sim.Environment(trace=False)
machine_list = list()
machine_dict = dict()
job_dict = dict()

for mach_id in range(3):
    machine = TestMachine(mach_id, env=env)
    machine_list.append(machine)
    machine_dict[mach_id] = machine

operation_procs = [8,10,3]
#job = TestJob(machine_list=machine_list, operation_procs=operation_procs, env=env, machine_dict=machine_dict)
#job = TestJob(ident=i, machine_list=machine_list, machine_dict=machine_dict, operation_procs=operation_procs,
                          #env=env, dispatcher=test_dispatcher)


for i in range(2):
    job_dict[i] = TestJob(ident=i, machine_list=machine_list, machine_dict=machine_dict, operation_procs=operation_procs, 
                          env=env, dispatcher=test_dispatcher)

test_dispatcher.add_dicts(job_dict, machine_dict)
env.run()

Job-ID 0 	 Operation with ID 0
Job-ID 1 	 Operation with ID 0
proc time of job with ID 0 on machine ID 0 is 8
proc time of job with ID 1 on machine ID 0 is 8
Time now Job-ID 0 is 8.0
Job-ID 0 	 Operation with ID 1
proc time of job with ID 0 on machine ID 1 is 10
Time now Job-ID 1 is 16.0
Job-ID 1 	 Operation with ID 1
proc time of job with ID 1 on machine ID 1 is 10
Time now Job-ID 0 is 18.0
Job-ID 0 	 Operation with ID 2
proc time of job with ID 0 on machine ID 2 is 3
Time now Job-ID 0 is 21.0
Time now Job-ID 1 is 28.0
Job-ID 1 	 Operation with ID 2
proc time of job with ID 1 on machine ID 2 is 3
Time now Job-ID 1 is 31.0


In [15]:
machine = machine_list[0]

In [16]:
machine.ispassive()

False

In [33]:
env.step()

Time now is 10.0
Operation with ID 1
Machine instance is TestMachine (testmachine.1)
